The serial part

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from poly_fun import noisy_even_polynomial_analysis as noise_poly

In [ ]:
## Do fitting to noisy polynomial data
## Function signature is:
### original_coeff, fitted_coeff, noisy_data = 
### noisy_even_polynomial_analysis(rand_seed, sym_domain, print_statements=False, plots=False)

start = time.time()
original_coeff, fitted_coeff, noisy_data = noise_poly(0, 10., print_statements=True, plots=True)
# original_coeff, fitted_coeff, noisy_data = noise_poly(0, 10.)
end = time.time()

In [ ]:
print(type(original_coeff), type(fitted_coeff), type(noisy_data))
print(original_coeff.shape, fitted_coeff.shape, noisy_data.shape)
print('execution time : ', end - start)
print('original coefficients : ', original_coeff)
print('fitted coefficients : ', fitted_coeff)

In [ ]:
## Repeat this analysis a bunch of times (in series)
num_trials = 250000
seeds = range(num_trials)
domain = 10.

## Initialize a list to store the results 
results = []

start = time.time()
## Loop over seeds 
for seed in seeds:
	original_coeff, fitted_coeff, noisy_data = noise_poly(seed, domain)
	results.append({"seed": seed,"original_coeff": original_coeff,\
	 "fitted_coeff": fitted_coeff,"noisy_data": noisy_data})

## Convert results to a Pandas DataFrame
df = pd.DataFrame(results)
end = time.time()

## Display the DataFrame
print(df)

print('execution time (serial) : ', end - start)


What does it look like if I run `top -u <MY_NETID>` on the node that my job is running on?

![a screenshot of a terminal running top](./imgs/serial_top.png "Series top")

In [ ]:
## Sanity check
print(type(results))
print(results[400]['seed'])
print(results[400]['original_coeff'].shape)
print(results[400]['fitted_coeff'].shape)
print(results[400]['noisy_data'].shape)

The parallel part

In [ ]:
import multiprocessing
from multiprocessing import Pool
import os

In [ ]:
## How many tasks (hopefully it is the same as what we got from SLURM)?
print("Number of cpus Python thinks you have : ", multiprocessing.cpu_count())

## How to get the number from SLURM environmental variable?
ncpus = int(os.environ["SLURM_NTASKS"])
print("Number of cpus you asked for from SLURM : ", ncpus)

In [ ]:
## Define a helper function for each parallel worker 
def worker(args):
	## Unpack the arguments
	seed, domain = args
	## Call the function that does the work
	original_coeff, fitted_coeff, noisy_data = noise_poly(seed, domain)
	## Reformat the output
	output = {"seed": seed,"original_coeff": original_coeff,\
	 "fitted_coeff": fitted_coeff,"noisy_data": noisy_data}
	## Return the output
	return output

## Create argument tuples 
arguments = [(seed, domain) for seed in seeds]

In [ ]:
# Parallel processing with Pool
start = time.time()
## ncpus should be the number of cores you asked SLURM for
with Pool(ncpus) as pool:
    results = pool.map(worker, arguments)

# Convert results to a Pandas DataFrame
df = pd.DataFrame(results)
end = time.time()

# Display the DataFrame
print(df)

print('execution time (serial) : ', end - start)

What does it look like if I run `top -u <MY_NETID>` on the node that my job is running on?

![a screenshot of a terminal running top](./imgs/parallel_top.png "Parallel top")

In [ ]:
## Sanity check
print(type(results))
print(results[0])

Results:

Using the scripts run_poly_analysis_parallel.py and multiprocessing_poly_submission.sh, I ran 250000 trials of the polynomial fitting with using multiprocessing to parallelize over different numbers of cores. Below are the results for script execution time vs. number of cores and parallel speedup vs. number of cores. Speedup(N) = (time to run script serially (with one core))/(time to run script parallelized over N cores). 

![a graph of parallel execution time](./imgs/parallel_execution_time.png "Parallel execution time")

![a graph of parallel speedup](./imgs/parallel_speedup.png "Parallel speedup")